In [1]:
import pickle, os
import requests as reqlib

import pandas as pd
import numpy as np
import geopandas as gpd

from tqdm.notebook import tqdm

In [2]:
survey_path = "../../results/surveys/egt_2010/cleaned"
spatial_path = "../../results/surveys/egt_2010/spatial.parquet"
parameters_path = "../../results/transit/calibration.p"

output_path = "../../results/transit/routing.parquet"

routing_endpoint = "http://localhost:8054/router/transit"
routing_endpoint = "http://localhost:8029/router/transit"

In [3]:
df_trips = pd.read_parquet("{}/trips.parquet".format(survey_path))
df_spatial = gpd.read_parquet(spatial_path)

In [4]:
# Prepare spatial data
df_spatial["origin_x"] = df_spatial["origin_geometry"].x
df_spatial["origin_y"] = df_spatial["origin_geometry"].y
df_spatial["destination_x"] = df_spatial["destination_geometry"].x
df_spatial["destination_y"] = df_spatial["destination_geometry"].y

# Only keep valid trips
df_trips = df_trips[df_trips["is_valid"]].copy()

# Merge in spatial data
df_trips = pd.merge(df_trips, df_spatial[[
    "trip_id",
    "origin_x", "origin_y",
    "destination_x", "destination_y"
]])

# Add request index
df_trips["request_index"] = np.arange(len(df_trips))

In [5]:
# Convert to requests
requests = []

for index, row in df_trips.iterrows():
    requests.append({
        "request_index": int(row["request_index"]),
        "origin_x": row["origin_x"],
        "origin_y": row["origin_y"],
        "destination_x": row["destination_x"],
        "destination_y": row["destination_y"],
        "departure_time_s": row["departure_time"]
    })

In [6]:
# Load utilities
with open(parameters_path, "rb") as f:
    parameters = pickle.load(f)

    objectives = [item["objective"] for item in parameters]
    best_index = np.argmin(objectives)

    utilities = parameters[best_index]["utilities"]

In [7]:
# Query results
maximum_batch_size = 400

df_response = []
batch_index = 0

with tqdm(total = len(requests)) as progress:
    while batch_index * maximum_batch_size < len(requests):
        batch = requests[batch_index * maximum_batch_size : (batch_index + 1) * maximum_batch_size]

        response = reqlib.post(routing_endpoint, json = {
            "batch": batch,
            "utilities": utilities
        })

        df_response.append(pd.DataFrame.from_records(response.json()))
        progress.update(len(batch))

        batch_index += 1

df_response = pd.concat(df_response)

  0%|          | 0/123223 [00:00<?, ?it/s]

In [8]:
df_response = pd.merge(df_response, df_trips[["request_index", "trip_id"]], on = "request_index")
df_response = df_response.drop(columns = ["request_index"])

In [9]:
df_response.to_parquet(output_path)